###### TEAM: 660 TEAM 7 <br> TEAM MEMBERS NAME: JANMEJAY MOHANTY, PRANITA KHAMKAR, DIVYA SAGAR REDDY, KRISHNA SAI ANJANA SUNKARA <br> COURSE: BIA 660 WS <br> COURSE NAME: WEB MINING <br> TEAM MEMBERS CWID'S: 20009315, 10476826, 10476472, 20008932 <br> TEAM MEMBERS EMAIL'S: JMOHANTY@STEVENS.EDU, PKHAMKAR@STEVENS.EDU, DREDDY4@STEVENS.EDU, ksunkara@stevens.edu <br> PROGRAM: Classification 2

In [11]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier


In [12]:

#read the reviews and their polarities from a given file
def loadData(fname):
    reviews=[]
    labels=[]
    f=open(fname)
    for line in f:
        review,rating=line.strip().split('\t')  
        reviews.append(review.lower())    
        labels.append(rating)
    f.close()
    return reviews,labels


In [13]:

rev_train,labels_train=loadData('reviews_train.txt')
rev_test,labels_test=loadData('reviews_test.txt')

#Build a counter based on the training dataset
#counter = CountVectorizer(stop_words=stopwords.words('english'))
counter = TfidfVectorizer(sublinear_tf=True,ngram_range=(1,2))
counter.fit(rev_train)

#count the number of times each term appears in a document and transform each doc into a count vector
counts_train = counter.transform(rev_train)#transform the training data
counts_test = counter.transform(rev_test)#transform the testing data

KNN_classifier=KNeighborsClassifier()
LREG_classifier=LogisticRegression(solver='liblinear')
DT_classifier = DecisionTreeClassifier()
SGD_classifier = SGDClassifier(tol=None)
SVM_classifier = svm.SVC(C=1.4, kernel='linear', gamma='auto')

predictors=[('knn',KNN_classifier),('lreg',LREG_classifier),('dt',DT_classifier),('sgd',SGD_classifier),('svm',SVM_classifier)]



In [14]:

#=======================================================================================
#build the parameter grid
KNN_grid = [{'n_neighbors': [1,3,5,7,9,11,13,15,17], 'weights':['uniform','distance']}]

#build a grid search to find the best parameters
gridsearchKNN = GridSearchCV(KNN_classifier, KNN_grid, cv=5)

#run the grid search
gridsearchKNN.fit(counts_train,labels_train)

#=======================================================================================

print(gridsearchKNN.best_params_)



{'n_neighbors': 11, 'weights': 'distance'}


In [15]:

#build the parameter grid
DT_grid = [{'max_depth': [3,4,5,6,7,8,9,10,11,12],'criterion':['gini','entropy']}]

#build a grid search to find the best parameters
gridsearchDT  = GridSearchCV(DT_classifier, DT_grid, cv=5)

#run the grid search
gridsearchDT.fit(counts_train,labels_train)

print(gridsearchDT.best_params_)



{'criterion': 'gini', 'max_depth': 7}


In [16]:

#=======================================================================================

#build the parameter grid
LREG_grid = [ {'C':[0.5,1,1.5,2],'penalty':['l1','l2']}]

#build a grid search to find the best parameters
gridsearchLREG  = GridSearchCV(LREG_classifier, LREG_grid, cv=5)

#run the grid search
gridsearchLREG.fit(counts_train,labels_train)

#=======================================================================================

print(gridsearchLREG.best_params_)



{'C': 2, 'penalty': 'l2'}


In [17]:
#=======================================================================================

#build the parameter grid
SGD_grid = [ {'loss':['log_loss','hinge'],'penalty':['l1','l2'], 'alpha':[0.1,0.01,0.001,0.0001], 'max_iter':[1000]}]

#build a grid search to find the best parameters
gridsearchSGD  = GridSearchCV(SGD_classifier, SGD_grid, cv=5)

#run the grid search
gridsearchSGD.fit(counts_train,labels_train)

#=======================================================================================

print(gridsearchSGD.best_params_)

{'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 1000, 'penalty': 'l2'}


In [18]:
#=======================================================================================

#build the parameter grid
SVM_grid = [ {'C':[0.5,1.2,1.4,1.6,1.8,2],'kernel':['linear','rbf'],'gamma':['scale','auto']}]

#build a grid search to find the best parameters
gridsearchSVM  = GridSearchCV(SVM_classifier, SVM_grid, cv=5)

#run the grid search
gridsearchSVM.fit(counts_train,labels_train)

#=======================================================================================

print(gridsearchSVM.best_params_)

{'C': 1.6, 'gamma': 'scale', 'kernel': 'linear'}


In [20]:
VT=VotingClassifier(predictors)

VT.fit(counts_train,labels_train)

#use the VT classifier to predict
predicted=VT.predict(counts_test)

#print the accuracy
print (accuracy_score(predicted,labels_test)*100)

92.0
